In [5]:
#Initialize sqlalchemy

from sqlalchemy import *
import datetime

In [6]:
## Initialize mediacloud
import mediacloud, json, datetime

MY_API_KEY = '9dacfeea9306ffccd9290e8efaed12bcab3e94d33a798944690240860c920eb9'
# from config import MY_API_KEY

mc = mediacloud.api.MediaCloud(MY_API_KEY)

keyword1 = 'trump'
res = mc.sentenceCount(keyword1, solr_filter=[mc.publish_date_query( datetime.date( 2016, 11, 1), datetime.date( 2017, 11, 30) ), 'tags_id_media:1' ])
query1 = {'keyword': keyword1, 'count': res['count']}
print(query1)

keyword2 = 'clinton'
res = mc.sentenceCount(keyword2, solr_filter=[mc.publish_date_query( datetime.date( 2016, 11, 1), datetime.date( 2017, 11, 30) ), 'tags_id_media:1' ])
query2 = {'keyword': keyword2, 'count': res['count']}
print(query2)


/Library/Python/2.7/site-packages/urllib3/util/ssl_.py:339: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  SNIMissingWarning
/Library/Python/2.7/site-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


{'count': 3168383, 'keyword': 'trump'}


/Library/Python/2.7/site-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


{'count': 338393, 'keyword': 'clinton'}


In [7]:
#Create a table 

# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
db_conn = engine.connect()
metadata = MetaData()
# define a table to use
keywords_table = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keyword', String(400), nullable=False),
    Column('count', Integer, nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the databa

2017-12-15 09:39:16,801 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-12-15 09:39:16,804 INFO sqlalchemy.engine.base.Engine ()
2017-12-15 09:39:16,808 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-12-15 09:39:16,809 INFO sqlalchemy.engine.base.Engine ()
2017-12-15 09:39:16,814 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2017-12-15 09:39:16,819 INFO sqlalchemy.engine.base.Engine ()
2017-12-15 09:39:16,823 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keyword VARCHAR(400) NOT NULL, 
	count INTEGER NOT NULL, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2017-12-15 09:39:16,828 INFO sqlalchemy.engine.base.Engine ()
2017-12-15 09:39:16,831 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
# Insert count per keyword
insert_stmt = keywords_table.insert().values([query1, query2])
result = db_conn.execute(insert_stmt)

2017-12-15 09:39:25,953 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keyword, count, timestamp) VALUES (?, ?, ?), (?, ?, ?)
2017-12-15 09:39:25,957 INFO sqlalchemy.engine.base.Engine ('trump', 3168383, '2017-12-15 09:39:25.953136', 'clinton', 338393, '2017-12-15 09:39:25.953157')
2017-12-15 09:39:25,961 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
# Select count per keyword
# Show total

from sqlalchemy.sql import select

select_stmt = select([keywords_table])
results = db_conn.execute(select_stmt)

total_count = 0

for row in results:
    total_count += row.count
    print('The count for {} is {}'.format(row.keyword, row.count))

print('Total count is {}'.format(total_count))

2017-12-15 09:39:34,717 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keyword, queries.count, queries.timestamp 
FROM queries
2017-12-15 09:39:34,719 INFO sqlalchemy.engine.base.Engine ()
The count for trump is 3168383
The count for clinton is 338393
Total count is 3506776
